# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
from dominate.svg import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
from typing import Iterable
import math

from python import constants, functions, espn_data

from IPython.core.display import HTML

# Testing and Debugging

In [6]:
seasons = [functions.summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Andrew']

data

,Week,Year,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking
7,None,2019,Andrew,6,7,6-7,1574.14,1598.96,121.09,102,123.00,104,-1.91,-7,0,8
4,None,2020,Andrew,6,7,6-7,1645.84,1545.26,126.60,104,118.87,98,7.74,-2,0,5
0,None,2021,Andrew,10,4,10-4,1901.54,1755.20,135.82,110,125.37,101,10.45,-3,1,1
3,None,2022,Andrew,8,6,8-6,1599.88,1650.06,114.28,100,117.86,104,-3.58,-4,0,4
3,None,2023,Andrew,8,6,8-6,1672.62,1532.50,119.47,102,109.46,94,10.01,2,0,4
1,None,2024,Andrew,10,4,10-4,1873.18,1681.98,133.80,113,120.14,102,13.66,-3,0,2


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [3]:
year = 2025
week = 2

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] == week)].copy()
data['Proj Diff'] = round(data['Points'] - data['Projected Points'], 2)

bench_df = data.loc[data['Slot Position'] == 'BE', ['Team','Player','Position','Points']].copy().sort_values('Points', ascending=False).reset_index(drop=True).head(5)
bench_table = functions.df_to_table(data=bench_df, table_id='topScoringBench')
bench_div = functions.content_container(title='Highest Scoring Bench Players', content=bench_table)

under_df = data.loc[data['Slot Position'] != 'BE', ['Team','Player','Position','Proj Diff']].copy().sort_values('Proj Diff', ascending=True).reset_index(drop=True).head(5)
under_table = functions.df_to_table(data=under_df, table_id='topUnderPerformers')
under_div = functions.content_container(title='Biggest Flops', content=under_table)

over_df = data.loc[data['Slot Position'] != 'BE', ['Team','Player','Position','Proj Diff']].copy().sort_values('Proj Diff', ascending=False).reset_index(drop=True).head(5)
over_table = functions.df_to_table(data=over_df, table_id='topOverPerformers')
over_div = functions.content_container(title='Biggest Successes', content=over_table)


display(HTML(str(bench_div)))
display(HTML(str(under_div)))
display(HTML(str(over_div)))

Team,Player,Position,Points
Ethan,Jared Goff,QB,33.96
Zach,Wan'Dale Robinson,WR,28.2
Andrew,Drake Maye,QB,26.3
Tyler,Jordan Love,QB,20.88
Ethan,Jauan Jennings,WR,19.9


Team,Player,Position,Proj Diff
Ethan,Derrick Henry,RB,-15.61
Kevin,Justin Fields,QB,-15.27
Kevin,Broncos D/ST,D/ST,-12.53
Nathan,Omarion Hampton,RB,-12.22
Dante,Courtland Sutton,WR,-12.13


Team,Player,Position,Proj Diff
Michael,Amon-Ra St. Brown,WR,22.28
Kevin,Malik Nabers,WR,19.37
Carter,Rome Odunze,WR,18.29
Andrew,Ja'Marr Chase,WR,16.64
Kevin,Brandon Aubrey,K,14.66


### Best Lineup table
### Includes Projection Hit/Miss

In [6]:
year = 2025
week = 3

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] == week)].copy()
data = data.sort_values('Points', ascending=False)

best_score = []

for team in data['Team'].unique():
    data_team = data.loc[data['Team'] == team].copy()
    actual = data_team.loc[data_team['Slot Position'] != 'BE', 'Points'].sum()
    proj = data_team.loc[data_team['Slot Position'] != 'BE', 'Projected Points'].sum()
    proj_diff = actual - proj

    score = 0
    for position in ['QB','TE','D/ST','K']:
        score += data_team.loc[data_team['Position'] == position, 'Points'].max()

    for position in ['RB','WR']:
        score += data_team.loc[data_team['Position'] == position, 'Points'].nlargest(2).sum()

    third_rb = data_team.loc[data_team['Position'] == 'RB', 'Points'].nlargest(3).iloc[-1]
    third_wr = data_team.loc[data_team['Position'] == 'WR', 'Points'].nlargest(3).iloc[-1]
    second_te = data_team.loc[data_team['Position'] == 'TE', 'Points'].nlargest(2).iloc[-1]

    score += max(third_rb, third_wr, second_te)

    coverage = round(actual/score * 100)

    best_score.append({'Team':team, 'Actual Score':actual, 'Projection Hit/Miss':proj_diff, 'Best Score':score, 'Pct Coverage':f'{coverage}%'})

pd.DataFrame(best_score).sort_values('Pct Coverage', ascending=False)


,Team,Actual Score,Projection Hit/Miss,Best Score,Pct Coverage
10,Noah,86.84,-33.64,93.24,93%
0,Michael,132.94,6.54,144.04,92%
3,Ethan,114.34,-12.85,125.54,91%
5,Dante,132.02,5.06,144.42,91%
7,Zach,118.30,-10.71,131.50,90%
6,Nathan,105.72,-7.30,126.42,84%
8,Andrew,92.62,-27.85,113.42,82%
2,Kevin,113.56,-9.98,146.56,77%
11,McGwire,69.38,-44.72,90.68,77%
1,Carter,116.86,-7.36,156.12,75%


### Score by Position Breakdown

In [5]:
year = 2025
# week = 1

data = constants.PLAYER_GAME_DATA.loc[(constants.PLAYER_GAME_DATA['Year'] == year) & (constants.PLAYER_GAME_DATA['Week'] <= 14)].copy()
data = data.loc[data['Slot Position'] != 'BE']

score_breakdown = []
for team in data['Team'].unique():
    score_line = {'Team':team}
    data_team = data.loc[data['Team'] == team].copy()

    full_score = data_team['Points'].sum()
    score_line['Full Score'] = full_score

    pos_scores = {}
    for position in ['QB','RB','WR','TE','D/ST','K']:
        pos_scores[position] = round(data_team.loc[data_team['Position'] == position, 'Points'].sum() / full_score * 100)
        score_line[f'{position} Points'] = pos_scores[position]


    score_breakdown.append(score_line)

pd.DataFrame(score_breakdown)

,Team,Full Score,QB Points,RB Points,WR Points,TE Points,D/ST Points,K Points
0,Kevin,364.56,10,24,43,7,5,12
1,Ethan,359.86,18,32,25,11,7,7
2,Zach,391.50,16,40,20,9,9,7
3,Tyler,316.38,9,22,48,5,6,9
4,McGwire,277.60,16,37,25,5,8,10
5,Andrew,289.80,25,24,22,14,7,8
6,Haris,333.50,12,39,29,5,7,8
7,Noah,272.50,15,27,40,10,0,9
8,Carter,379.06,16,28,30,8,11,7
9,Nathan,294.30,20,25,31,11,6,6
